In [4]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (13.9s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [20]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

COUNT_YEARS = 6
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", "ETHUSDT-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [21]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: 2024-02-05 08:00:00+00:00 (58 hours left)
current price: 2300.6
-------------------------------
trading put 2275 and call 2325 (hit ratio: 61%)
cost: 0.007 / value: 0.029, benefit/cost = 4.1x
($) cost: 16.56 / value: 67.77, average gain = 51.21
-------------------------------
trading put 2250 and call 2350 (hit ratio: 51%)
cost: 0.004 / value: 0.022, benefit/cost = 6.2x
($) cost: 8.28 / value: 51.66, average gain = 43.38
-------------------------------
trading put 2225 and call 2375 (hit ratio: 40%)
cost: 0.002 / value: 0.017, benefit/cost = 9.1x
($) cost: 4.37 / value: 39.56, average gain = 35.19
-------------------------------
trading put 2200 and call 2400 (hit ratio: 31%)
cost: 0.001 / value: 0.013, benefit/cost = 12.0x
($) cost: 2.53 / value: 30.43, average gain = 27.90


[{'put': 2250.0, 'call': 2350.0, 'quantity': 2},
 {'put': 2225.0, 'call': 2375.0, 'quantity': 3}]

In [5]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2175.0,104.585638,NaN,NaN,13.248727,0.005846,0.0017
2200.0,84.031209,NaN,NaN,17.694297,0.007808,0.0032
2225.0,65.111732,NaN,NaN,23.774820,0.010491,0.0055
2250.0,48.653754,NaN,0.0165,32.316843,0.014260,0.0090
2275.0,35.764609,0.015782,0.0105,44.427698,NaN,0.0140
2300.0,26.473994,0.011682,0.0065,60.137083,NaN,0.0225
2325.0,19.662775,0.008676,0.0041,78.325864,NaN,0.0320
2350.0,14.653544,0.006466,0.0025,98.316633,NaN,NaN
2375.0,10.958756,0.004836,0.0017,119.621845,NaN,NaN


In [22]:
import binanceprices
test2 = binanceprices.load_prices(s3, "test-binance-prices-255120844515", "ETHUSDT", 6, no_update=False)
test2

,open,high,low,close,volume
dateTime,,,,,
2018-01-01 00:00:00,733.01,734.52,720.03,727.62,2105.90100
2018-01-01 01:00:00,727.01,732.00,716.80,717.97,2305.97086
2018-01-01 02:00:00,717.67,725.75,717.59,724.05,2166.45725
2018-01-01 03:00:00,723.95,737.99,722.70,734.50,2160.90450
2018-01-01 04:00:00,734.99,744.98,730.01,744.82,2335.33705
...,...,...,...,...,...
2024-01-31 19:00:00,2335.63,2342.26,2309.51,2323.38,24021.48830
2024-01-31 20:00:00,2323.39,2324.67,2286.11,2287.98,27445.83220
2024-01-31 21:00:00,2287.98,2292.37,2263.57,2279.00,23011.28190


In [23]:
prices_df

,open,high,low,close,volume
dateTime,,,,,
2018-01-01 00:00:00,733.01,734.52,720.03,727.62,2105.90100
2018-01-01 01:00:00,727.01,732.00,716.80,717.97,2305.97086
2018-01-01 02:00:00,717.67,725.75,717.59,724.05,2166.45725
2018-01-01 03:00:00,723.95,737.99,722.70,734.50,2160.90450
2018-01-01 04:00:00,734.99,744.98,730.01,744.82,2335.33705
...,...,...,...,...,...
2024-01-31 19:00:00,2335.63,2342.26,2309.51,2323.38,24021.48830
2024-01-31 20:00:00,2323.39,2324.67,2286.11,2287.98,27445.83220
2024-01-31 21:00:00,2287.98,2292.37,2263.57,2279.00,23011.28190
